In [1]:
import sys
sys.path.insert(0, "/kaggle/input/transformer-by-abhinav")

In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/transformer-by-abhinav/spa.txt", sep="\t", header=None, names=["English", "Spanish", "Attribution"], encoding="utf-8")

print(df.head())


  English  Spanish                                        Attribution
0     Go.      Ve.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1     Go.    Vete.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2     Go.    Vaya.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3     Go.  Váyase.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4     Hi.    Hola.  CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [3]:
df.shape

(139013, 3)

In [4]:
l_eng = []
for i in df['English']:
    l_eng.append(len(i.split(" ")))
max(l_eng)

70

In [5]:
l_spa = []
for i in df['Spanish']:
    l_spa.append(len(i.split(" ")))
max(l_spa)

68

In [6]:
import re
en_sent = df['English']
sp_sent = df['Spanish']
en_sent = en_sent.apply(lambda x: x.lower())
sp_sent = sp_sent.apply(lambda x: x.lower())


In [7]:
en_sent = en_sent.apply(lambda x: re.sub(r'[,.!?¿¡]', '', x))
sp_sent = sp_sent.apply(lambda x: re.sub(r'[,.!?¿¡]', '', x))

In [8]:
sp_sent = sp_sent.apply(lambda x: '<sos>' + x + '<eos>')

In [9]:
from encoder import Encoder
en_encoder = Encoder(en_sent,max_len = 100)
sp_encoder = Encoder(sp_sent,max_len = 100)

In [10]:
en_encoder.build_vocab()
len(en_encoder.vocab)

10000

In [11]:
sp_encoder.build_vocab()
len(sp_encoder.vocab)

10000

In [12]:
en_encoded = []
for i in en_sent.values:
    en_encoded.append(en_encoder.encode_text(i))

In [13]:
sp_encoded = []
for i in sp_sent.values:
    sp_encoded.append(sp_encoder.encode_text(i))

In [14]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from model import Transformer
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(10000,10000,128,2,6,2048,100,0.1).to(device)


In [16]:
import torch
import torch.nn as nn
import time
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (0.9,0.98),eps = 1e-9)

en_encoded = torch.tensor(en_encoded)
sp_encoded = torch.tensor(sp_encoded)
batch_size = 128

num_samples = en_encoded.size(0)
num_batches = num_samples // batch_size

for epoch in range(50):
    total_loss = 0
    start = time.time()

    for i in range(0, num_samples, batch_size):
        src_batch = en_encoded[i:i+batch_size]
        tgt_batch = sp_encoded[i:i+batch_size]
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        # Teacher forcing: decoder input is tgt[:, :-1]
        tgt_input = tgt_batch[:, :-1]

        # Model output
        output = model(src_batch, tgt_input)   # (batch, seq, vocab_size)

        # Target shifted by 1
        tgt_output = tgt_batch[:, 1:].contiguous().view(-1)

        # Reshape output for loss
        output = output.contiguous().view(-1, 10000)

        loss = criterion(output, tgt_output)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print("Training Started")
    print(f"epoch: {epoch+1}, loss: {total_loss/num_batches:.4f}")
    print(f"epoch took:{time.time()-start} seconds")
print("Training has ended")


/tmp/ipykernel_47/2914600894.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  en_encoded = torch.tensor(en_encoded)
/tmp/ipykernel_47/2914600894.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sp_encoded = torch.tensor(sp_encoded)


Training Started
epoch: 1, loss: 4.1201
epoch took:195.63371753692627 seconds
Training Started
epoch: 2, loss: 3.6500
epoch took:195.54999423027039 seconds
Training Started
epoch: 3, loss: 3.2801
epoch took:195.4784643650055 seconds
Training Started
epoch: 4, loss: 2.9850
epoch took:195.39964699745178 seconds
Training Started
epoch: 5, loss: 2.7494
epoch took:195.27919101715088 seconds
Training Started
epoch: 6, loss: 2.5490
epoch took:195.24465322494507 seconds
Training Started
epoch: 7, loss: 2.3767
epoch took:195.38073134422302 seconds
Training Started
epoch: 8, loss: 2.2336
epoch took:195.36984634399414 seconds
Training Started
epoch: 9, loss: 2.1073
epoch took:195.03599762916565 seconds
Training Started
epoch: 10, loss: 1.9954
epoch took:194.96253967285156 seconds
Training Started
epoch: 11, loss: 1.8964
epoch took:195.1979911327362 seconds
Training Started
epoch: 12, loss: 1.8055
epoch took:195.30326390266418 seconds
Training Started
epoch: 13, loss: 1.7281
epoch took:195.1593885

In [18]:
torch.save(model.state_dict,"/kaggle/working/model.pth")
import pickle
with open("/kaggle/working/en_encoder.pkl",'wb') as f:
    pickle.dump(en_encoder,f)
with open("/kaggle/working/sp_encoder",'wb') as f:
    pickle.dump(sp_encoder,f)